In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
class Plot:
    def __init__(self, file, names):        
        n_all = [pd.read_csv('stats/node_{}/{}'.format(node, file), sep=',', header=None,
                             names=names+['time_us']) for node in [1,2,3]]
        
        x = pd.concat(n_all)   
        
        x = x[x['numf'] < 120].reset_index(drop=True)
        
        steps = x['epochs']*x['dataset size']/x['batch']

        x['time_ms'] = x['time_us']/steps/1000
        
        self.df = x.drop(columns=['epochs', 'dataset size', 'time_us'])
        
    def plot(self, feat, title=None):
        dfs = [self.df[self.df['nodes']==node] for node in [1,2,3]]
        
        def org(df):
            return df[[feat, 'time_ms']].groupby(by=[feat]).mean().reset_index()
        
        scale = 2
        plt.figure(figsize=[scale*6.4, scale*4.8])
        
        colors = ['r','g','b']
        lines = [3,3,1]
        for j,marker in enumerate(['x','+','o']):
            _df = org(dfs[j])
            plt.scatter(_df[feat], _df['time_ms'], label='nodes: {}'.format(j+1), marker=marker,
                        s=100,
                        color=colors[j], 
                        linewidths=lines[j] 
                       )
            
        plt.legend(fontsize=20)
        plt.xlabel(feat if title == None else title, fontsize=20)
        plt.ylabel('Μέσος Χρόνος Εκτέλεσης (ms)', fontsize=20)
        plt.tight_layout()
    
    def _check(self, dim, fw):
        assert dim in [1,2]
        assert fw in ['tflow', 'ptorch']
        
    def all_names(self):
        return ['epochs', 'dataset size', 'numf', 'channels', 'batch', 'nodes']
    
    def dense_names(self):
        return ['epochs', 'dataset size', 'numf', 'batch', 'nodes', 'units']
        

In [ ]:
class Conv(Plot):
    def __init__(self, dim, fw):
        super()._check(dim, fw)
        
        self.file = 'conv{}d.{}'.format(dim, fw)
        super().__init__(self.file, super().all_names() + ['kernel', 'stride', 'filters'])

In [ ]:
class Pool(Plot):
    def __init__(self, typ, dim, fw):
        assert typ in ['avg', 'max']
        super()._check(dim, fw)
        
        self.file = '{}{}d.{}'.format(typ, dim, fw)
        super().__init__(self.file, super().all_names() + ['pool', 'stride'])

In [ ]:
class Drop(Plot):
    def __init__(self, dim, fw):
        super()._check(dim, fw)
        self.file = 'drop{}d.{}'.format(dim, fw)
        
        super().__init__(self.file, super().all_names() + ['kernel', 'drop'])

In [ ]:
class Rest(Plot):
    def __init__(self, typ, dim, fw):
        assert typ in ['norm', 'relu', 'tanh', 'flatten']
        super()._check(dim, fw)
        assert not ((typ=='flatten') & (fw=='tflow'))
        
        self.file = '{}{}d.{}'.format(typ, dim, fw)
        super().__init__(self.file, super().all_names())

# PyTorch